In [5]:
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch()

In [7]:
try:
    es.indices.delete('yowl')
except:
    pass

body = {
    "mappings": {
      "restaurants": {
        "properties": {
          "location": {
            "type": "geo_point"}}}}}
es.indices.create('yowl', body)

{u'acknowledged': True}

### Content

In [40]:
body = {
    'query': {
        'multi_match': {
            'query': 'hello',
            'fields': ['name^10', 'cuisine_high_q^10', 'cuisine_low_q^4', 'menu^2', 'description^1'],
        },
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 30}

### Customer Preferences

In [41]:
body = { 
    'query': {
        'bool': {
            'filter': [
                {'terms':{
                    'price': ['$$','$$$','$$$$'],
                }},
                {'terms':{
                    'rating': ['☆☆☆','☆☆☆☆'],
                }}
            ]
        }
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 3}

### Location

In [42]:
body = {
    'query': {
        'function_score': {
            'functions': [{
                'gauss': {
                    'location': { 
                        'origin': {
                                'lat': 36.154547,
                                'lon': -86.782277
                        },
                        'offset': '0km',
                        'scale':  '10km'
                    }
                }
            }
        ]}
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'AVAvpMcz9qTstewm_Jve',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'has_discount': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqAba9qTstewm_Jvi',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'has_discount': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqwt99qTstewm_Jvp',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvpC1C9qTstewm_Jvc',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'has_discount': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvpSZF9qTstewm_Jvf',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'has_discount': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqJbn9qTstewm_Jvj',
    u'_index': u'yowl',
    u'_score': 1.0,
    u'_source': {u'has_discount': True}

### Business

In [43]:
body = {
    'has_discount': True,
    'promoted': True,
    'engaged': True,
}
es.index('yowl','restaurants',body)
body = {
    'query': {
        'function_score': {
            'functions': [{
                'filter' : {'term':{'has_discount': True}},
                'weight': 4
            },
            {
                'filter' : {'term':{'promoted': True}},
                'weight': 3
            },
            {
                'filter' : {'term':{'engaged': True}},
                'weight': 2
            }
        ]}
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'AVAvqwt99qTstewm_Jvp',
    u'_index': u'yowl',
    u'_score': 24.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvtII_9qTstewm_Jvt',
    u'_index': u'yowl',
    u'_score': 24.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvtW8c9qTstewm_Jvu',
    u'_index': u'yowl',
    u'_score': 24.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvsnr09qTstewm_Jvr',
    u'_index': u'yowl',
    u'_score': 24.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqg8P9qTstewm_Jvk',
    u'_index': u'yowl',
    u'_score': 24.0,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True}

### Combined

In [8]:
body = {
    'query': {
        'function_score': {
            'query': {
                'multi_match': {
                    'query': 'hello',
                    'fields': ['name^10', 'cuisine_high_q^10', 'cuisine_low_q^4', 'menu^2', 'description^1'],
                }
            },   
        }
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 4}

## Scraps

In [30]:
# possible alternative to the business query
body = {
    'has_discount': True,
    'promoted': True,
    'engaged': True,
}
es.index('yowl','restaurants',body)
body = {
    'query': {
        'function_score': {
            'functions': [{
                'script_score' : {
                    'script' : """
                        (doc['has_discount'].value*2)+
                        (doc['promoted'].value*3.5)+
                        (doc['engaged'].value*5)"""
                }
            }
        ]}
    }
}
es.search('yowl',body=body)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'AVAvqg8P9qTstewm_Jvk',
    u'_index': u'yowl',
    u'_score': 10.5,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqn_i9qTstewm_Jvn',
    u'_index': u'yowl',
    u'_score': 10.5,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqkno9qTstewm_Jvl',
    u'_index': u'yowl',
    u'_score': 10.5,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqlfY9qTstewm_Jvm',
    u'_index': u'yowl',
    u'_score': 10.5,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True},
    u'_type': u'restaurants'},
   {u'_id': u'AVAvqsQ09qTstewm_Jvo',
    u'_index': u'yowl',
    u'_score': 10.5,
    u'_source': {u'engaged': True, u'has_discount': True, u'promoted': True}